In [1]:
import pandas as pd
import numpy as np
from importnb import Notebook
import sys, io
import matplotlib
import glob, os
# add the folder vonheijine that contains our vonheijine functions
sys.path.append('../Feature_Selection/')
# avoid to print useless things when importing another notebook
matplotlib.use('Agg')  # blocca la visualizzazione grafica
old_stdout = sys.stdout
sys.stdout = io.StringIO()  

with Notebook():
    import custom_features, feauture_selection
sys.stdout = old_stdout

#remove temporary files created by feature_selection
for f in glob.glob("*_features_*.npz"):
    os.remove(f)

/home/mc02/lab2_project/LB2_project_Group_3/performance_SVM/../Feature_Selection/feauture_selection.ipynb:607: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/home/mc02/lab2_project/LB2_project_Group_3/performance_SVM/../Feature_Selection/feauture_selection.ipynb:646: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/home/mc02/lab2_project/LB2_project_Group_3/performance_SVM/../Feature_Selection/feauture_selection.ipynb:607: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/home/mc02/lab2_project/LB2_project_Group_3/performance_SVM/../Feature_Selection/feauture_selection.ipynb:646: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/home/mc02/lab2_project/LB2_project_Group_3/performance_SVM/../Feature_Selection/feauture_selection.ipynb:607: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/home/mc02

In [2]:
print(feauture_selection.features_to_use)

['VhonHeijne', 'C', 'tm_tendency_max', 'chou_fasman_h_mean', 'max_miyazawa_mean', 'D', 'T', 'R', 'chou_fasma_b_max', 'N', 'flexibility_max', 'punta_max', 'bulkiness_mean', 'M', 'argos_max']


In [16]:
def update_vonheijne(sets, matrix): 
    seq_features=[]
    for seq in sets:
        seq=seq.replace("X" , "")
        seq=seq.replace("U" , "C")
        vonhejine=custom_features.vonheijne_feature(matrix, seq) #get the von heijne feature for that sequence
        seq_features.append(vonhejine)
    hejine_col = np.array(seq_features) #transform the list that contains all the features in an array
    return hejine_col

In [17]:
#Load the npz files of training, testing and validation sets for each iteration
# load training
# 5th iteration was: validation set 1 , training set 2,3,4 , testing set 5
loaded_data_train = np.load('../Feature_Selection/training_features_5.npz')
x_train = loaded_data_train['matrix']
y_train = loaded_data_train['target']

# load test
loaded_data_test = np.load('../Feature_Selection/testing_features_5.npz')
x_test = loaded_data_test['matrix']
y_test = loaded_data_test['target']

# load validation
loaded_data_validation = np.load('../Feature_Selection/validation_features_5.npz')
x_validation = loaded_data_validation['matrix']
y_validation = loaded_data_validation['target']

#concatenate the matrices in the correct order
x_training_conc = np.concatenate((x_validation, x_train , x_test), axis=0) #order is maintained: 1,2,3,4,5
y_training_conc = np.concatenate((y_validation , y_train, y_test), axis=0)


In [18]:
#Load the benchmark set and encode it
dataset = pd.read_csv("../Data_Preparation/train_bench.tsv", sep = "\t")
benchmark=dataset.query("Set=='Benchmark'")
training= pd.concat([dataset.query("Set=='2'"), dataset.query(" Set=='3' or Set=='4' or Set=='5' "), dataset.query("Set=='1'")],axis=0,ignore_index=True)
matrix_training=custom_features.get_pswm(training , 13 , 2)
#replace the old VonHejine feature with the new VonHejine basing on the updated PSWM
x_training_conc[:, 17] = update_vonheijne(training["Sequence"], matrix_training)
feature_set_benchmark , feature_order_training = custom_features.get_all_features(benchmark["Sequence"] , matrix_training, 15 )
vector_neg_pos = benchmark["Class"]
vector_proper = vector_neg_pos.map({"Positive": 1, "Negative": 0})
target_benchmark_vector = vector_proper.to_numpy()

In [19]:
index=[]
for feature in feature_order_training:
    if feature not in feauture_selection.features_to_use:
        index.append(feature_order_training.index(feature))
x_training_conc=np.delete(x_training_conc, index, axis=1)
x_benchmark=np.delete(feature_set_benchmark, index, axis=1)
np.savez('benchmark_features.npz', matrix=x_benchmark, target=target_benchmark_vector)
np.savez('training_features.npz', matrix=x_training_conc, target=y_training_conc)

## Hyperparameter Tuning

In [20]:
import numpy as np
from sklearn.metrics import matthews_corrcoef
#! pip install scikit-optimize
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import matthews_corrcoef , confusion_matrix

In [21]:
#create the pipeline
pipeline = Pipeline([("scaler" , StandardScaler()) , ("svm" , SVC(cache_size=1500))])

In [22]:
# parameter search using BayesSearchCV

# search space for rbf kernel
search_space = {
        "svm__kernel": Categorical(["rbf"]),
        "svm__C": Real(0.01, 100, prior="log-uniform"),                
        "svm__gamma": Real(0.01, 100, prior="log-uniform"), 
    }


In [23]:
#set up the BayesSearch
bayes = BayesSearchCV(
    estimator=pipeline,
    search_spaces=search_space,
    scoring="matthews_corrcoef",   
    n_jobs=-1,
    refit=False,                 
    random_state=42,
    verbose=2,
    cv=5,
    n_iter=60
)
bayes.fit(x_training_conc, y_training_conc)  # here we perform the bayes search

print("\n[Best parameters found:] ")
print(bayes.best_params_)
print(f"[Best MCC validation] {bayes.best_score_:.4f}")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END svm__C=0.4369339947510315, svm__gamma=8.145222883402798, svm__kernel=rbf; total time=   4.8s
[CV] END svm__C=0.6016307829589929, svm__gamma=47.30315267320388, svm__kernel=rbf; total time=   7.6s
[CV] END svm__C=17.76576664980768, svm__gamma=0.04869521028037488, svm__kernel=rbf; total time=   0.4s
[CV] END svm__C=15.783879853890564, svm__gamma=0.5650888599199564, svm__kernel=rbf; total time=   4.3s
[CV] END svm__C=8.632012725909878, svm__gamma=57.2109224721

In [24]:
#predict the benchmark set
pipeline.set_params(**bayes.best_params_).fit(x_training_conc, y_training_conc) #parameters of BayesSearchCV
bench_pred = pipeline.predict(x_benchmark)
#compute the mcc
mcc_bayes = matthews_corrcoef(target_benchmark_vector , bench_pred)
print(f"MCC on testing set (bayesian search): {mcc_bayes}")

MCC on testing set (bayesian search): 0.8081691772560898


In [15]:
conf_mat = confusion_matrix(target_benchmark_vector , bench_pred)
print(conf_mat)

[[1749   38]
 [  37  182]]
